In [ ]:
import torch
import pandas as pd

from tqdm import tqdm
from sae_lens import SAE
from matplotlib import pyplot as plt
from transformers import AutoTokenizer, AutoModelForCausalLM

%matplotlib inline
%config InlineBackend.figure_format='retina'

In [10]:
notebook_login()

Привет. Я попытался сделать задание через API нейропедии, но не мог получить ответы от модели поскольку постоянно упирался в лимит токенов. Я поднял диписик на A100 и использовал SAE с нейропедии для стиринга

In [4]:
fractions_df = pd.read_json('../fractions/fractions.json')

In [15]:
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Llama-8B")
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Llama-8B")

Loading checkpoint shards: 100%|██████████| 2/2 [00:37<00:00, 18.92s/it]


In [23]:
df = pd.read_json('../fractions/fractions.json')
df['questions'] = df['questions'].map(lambda x: (x.replace("What is ", "Solve folowing example. Write  only fraction.\n").replace("?", " = ")))

In [ ]:
conversations = []

for message in df['questions'].tolist():
  conversation = []
  conversation.append({
        "role": "user",
        "content": message
  })
  conversations.append(conversation)

In [ ]:
chat_conv_formatted = tokenizer.apply_chat_template(conversations, tokenize=False, add_generation_prompt=True)

inputs = tokenizer(chat_conv_formatted, return_tensors="pt", padding=True)
inputs.to("cuda:0")

In [24]:
sae, cfg_dict, sparsity = SAE.from_pretrained(
    release="llama_scope_r1_distill",
    sae_id="l15r_800m_slimpajama",
    device="cuda:0")


def generate_with_sae(model,
                      sae: SAE,
                      layer: int,
                      feature_index: int,
                      alpha: float,
                      do_sample: bool=True,
                      temperature: float=0.6,
                      max_len: int=512,
                      repetition_penalty:float=1.0,
                      normalize_new_hidden=False,
                      eos_token_id=128001):

    assert alpha >= 0

    def gather_act_hook(mod, inputs, outputs):

        h = outputs[0]

        try:

            h = sae.encode(h)
            steering = torch.zeros_like(h)
            steering[:, :, feature_index] = alpha

            if normalize_new_hidden:
                h_steer = h + steering
                h_steer = (h_steer / h_steer.norm(dim=-1, keepdim=True)) * h.norm(dim=-1, keepdim=True)
            else:
                h_steer = h + steering
        except:
            print("error")
            h_steer = outputs[0]

        h_steer = sae.decode(h_steer)

        return (h_steer,)

    hook = model.model.layers[layer].register_forward_hook(gather_act_hook)

    model = model.to("cuda:0")
    model.eval()
    with torch.inference_mode():
      outputs = model.generate(**inputs,
                              max_length=max_len,
                              temperature=temperature,
                              do_sample=do_sample,
                              repetition_penalty=repetition_penalty,
                              use_cache=False,
                              eos_token_id=eos_token_id)
    hook.remove()

    return outputs

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.

KeyboardInterrupt



In [ ]:
inputs = inputs.to("cuda:0")
steer_strengths = [5, -5, 1, -1, 0, -10, 10]

for steer_strength in tqdm(steer_strengths):

  encoded_outputs = generate_with_sae(
      model=model,
      sae=sae,
      alpha=steer_strength,
      layer=15,
      feature_index=30939,
  )

  decoded_outputs = []
  for output in encoded_outputs:
    decoded_outputs.append(tokenizer.decode(output, skip_special_tokens=False))

  str_steer_strength = str(steer_strength).replace("-", "minus_")
  df[f'answers_for_steer_{str_steer_strength}'] = decoded_outputs